In [ ]:
# I have performed Ex2-2 in this file & go below you will find the Ex2-2 questions which is performed for all the questions from Ex2-2 from cell 9.

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!tar xf spark-3.5.2-bin-hadoop3.tar
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.2-bin-hadoop3"

In [3]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [4]:
# Read the dataset into a Spark DataFrame named df
df = spark.read.csv("geolocation.csv", header=True, inferSchema=True)

In [5]:
df.show(5)

+-------+--------+---------+---------+-----------+----------+----------+--------+---------+----------+
|truckid|driverid|    event| latitude|  longitude|      city|     state|velocity|event_ind|idling_ind|
+-------+--------+---------+---------+-----------+----------+----------+--------+---------+----------+
|    A54|     A54|   normal|38.440467|-122.714431|Santa Rosa|California|      17|        0|         0|
|    A20|     A20|   normal|36.977173|-121.899402|     Aptos|California|      27|        0|         0|
|    A40|     A40|overspeed|37.957702| -121.29078|  Stockton|California|      77|        1|         0|
|    A31|     A31|   normal|39.409608|-123.355566|   Willits|California|      22|        0|         0|
|    A71|     A71|   normal|33.683947|-117.794694|    Irvine|California|      43|        0|         0|
+-------+--------+---------+---------+-----------+----------+----------+--------+---------+----------+
only showing top 5 rows



In [6]:
df.printSchema()

root
 |-- truckid: string (nullable = true)
 |-- driverid: string (nullable = true)
 |-- event: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- velocity: integer (nullable = true)
 |-- event_ind: integer (nullable = true)
 |-- idling_ind: integer (nullable = true)



In [7]:
df.select("truckid", "driverid", "event", "city").filter("event != 'normal'").show()

+-------+--------+--------------------+-------------+
|truckid|driverid|               event|         city|
+-------+--------+--------------------+-------------+
|    A40|     A40|           overspeed|     Stockton|
|     A5|      A5|unsafe following ...|      Klamath|
|    A48|     A48|           overspeed|    Roseville|
|    A94|     A94|unsafe following ...|  San Quentin|
|    A49|     A49|      lane departure|        Ukiah|
|    A97|     A97|           overspeed|   Santa Rosa|
|    A75|     A75|      lane departure|    San Dimas|
|     A9|      A9|unsafe tail distance|    San Diego|
|    A80|     A80|unsafe tail distance|San Francisco|
|    A91|     A91|unsafe following ...|   Santa Rosa|
|    A59|     A59|unsafe following ...| Santa Monica|
|    A85|     A85|      lane departure|San Francisco|
|    A94|     A94|           overspeed| San Fernando|
|    A73|     A73|           overspeed|  Bakersfield|
|    A71|     A71|      lane departure|       Irvine|
|    A28|     A28|      lane

In [8]:
df.groupBy("event").count().show()

+--------------------+-----+
|               event|count|
+--------------------+-----+
|      lane departure|  152|
|unsafe tail distance|   65|
|              normal| 7543|
|unsafe following ...|  150|
|           overspeed|   90|
+--------------------+-----+



In [9]:
df.describe("velocity").show()

+-------+------------------+
|summary|          velocity|
+-------+------------------+
|  count|              8000|
|   mean|           35.0565|
| stddev|24.863750391090154|
|    min|                 0|
|    max|               100|
+-------+------------------+



In [20]:
###Hands-On Experiment 2-2:
###Data Warehousing with Hive

In [10]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [11]:
!wget https://downloads.apache.org/hadoop/common/hadoop-3.3.6/hadoop-3.3.6.tar.gz
!tar -xzf hadoop-3.3.6.tar.gz
!tar xf apache-hive-3.1.3-bin.tar

--2025-03-03 01:43:56--  https://downloads.apache.org/hadoop/common/hadoop-3.3.6/hadoop-3.3.6.tar.gz
Resolving downloads.apache.org (downloads.apache.org)... 88.99.208.237, 135.181.214.104, 2a01:4f8:10a:39da::2, ...
Connecting to downloads.apache.org (downloads.apache.org)|88.99.208.237|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 730107476 (696M) [application/x-gzip]
Saving to: ‘hadoop-3.3.6.tar.gz’

hadoop-3.3.6.tar.gz 100%[===================>] 696.28M  81.3MB/s    in 11s     

2025-03-03 01:44:07 (64.1 MB/s) - ‘hadoop-3.3.6.tar.gz’ saved [730107476/730107476]



In [12]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["HADOOP_HOME"] = "/content/hadoop-3.3.6"
os.environ["HIVE_HOME"] = "/content/apache-hive-3.1.3-bin"

In [13]:
!rm -rf metastore_db

In [14]:
!$HIVE_HOME/bin/schematool -initSchema -dbType derby


SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Metastore connection URL:	 jdbc:derby:;databaseName=metastore_db;create=true
Metastore Connection Driver :	 org.apache.derby.jdbc.EmbeddedDriver
Metastore connection User:	 APP
Starting metastore schema initialization to 3.1.0
Initialization script hive-schema-3.1.0.derby.sql

 
 
 
 
 
 
 
 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 

 
 
 
 
 
 
 
 

 

 
 
 

 
 
 


 
 
 

 
 
 


 
 
 
 
 
 
 


 
 
 

 
 
 

 
 

 
 
 
 
 

 
 
 
 
 
 


In [15]:
!$HIVE_HOME/bin/hive -e 'show databases;'

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = a34fe5fc-25d9-45a1-986d-9ec8d7e17ff1

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = f9eb9a38-a5da-4606-854b-3845e1768d48
OK
default
Time taken: 1.569 seconds, Fetched: 1 row(s)


In [16]:
!$HIVE_HOME/bin/hive -e 'use default;'

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = 5a9718a1-1387-4209-82a9-242b87f38df8

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = 67d4a9d8-b81d-4088-8202-3d7062dce325
OK
Time taken: 2.094 seconds


In [17]:
%%bash
$HIVE_HOME/bin/hive -e "
CREATE TABLE default.amaht_trucks
(driverid String,truckid String,model String,
jun13_miles int,jun13_gas int,may13_miles int,may13_gas int,apr13_miles int,
apr13_gas int,mar13_miles int,mar13_gas int,feb13_miles int,feb13_gas int,
jan13_miles int,jan13_gas int,dec12_miles int,dec12_gas int,nov12_miles int,
nov12_gas int,oct12_miles int,oct12_gas int,sep12_miles int,sep12_gas int,
aug12_miles int,aug12_gas int,jul12_miles int,jul12_gas int,jun12_miles int,
jun12_gas int,may12_miles int,may12_gas int,apr12_miles int,apr12_gas int,
mar12_miles int,mar12_gas int,feb12_miles int,feb12_gas int,jan12_miles int,
jan12_gas int,dec11_miles int,dec11_gas int,nov11_miles int,nov11_gas int,
oct11_miles int,oct11_gas int,sep11_miles int,sep11_gas int,aug11_miles int,
aug11_gas int,jul11_miles int,jul11_gas int,jun11_miles int,jun11_gas int,
may11_miles int,may11_gas int,apr11_miles int,apr11_gas int,mar11_miles int,
mar11_gas int,feb11_miles int,feb11_gas int,jan11_miles int,jan11_gas int,
dec10_miles int,dec10_gas int,nov10_miles int,nov10_gas int,oct10_miles int,
oct10_gas int,sep10_miles int,sep10_gas int,aug10_miles int,aug10_gas int,
jul10_miles int,jul10_gas int,jun10_miles int,jun10_gas int,may10_miles int,
may10_gas int,apr10_miles int,apr10_gas int,mar10_miles int,mar10_gas int,
feb10_miles int,feb10_gas int,jan10_miles int,jan10_gas int,dec09_miles int,
dec09_gas int,nov09_miles int,nov09_gas int,oct09_miles int,oct09_gas int,
sep09_miles int,sep09_gas int,aug09_miles int,aug09_gas int,jul09_miles int,
jul09_gas int,jun09_miles int,jun09_gas int,may09_miles int,may09_gas int,
apr09_miles int,apr09_gas int,mar09_miles int,mar09_gas int,feb09_miles int,
feb09_gas int,jan09_miles int,jan09_gas int)
COMMENT 'trucks'
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
LINES TERMINATED BY '\n'
STORED AS TEXTFILE
TBLPROPERTIES('skip.header.line.count'='1');
"

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = ec9e0841-950b-4450-9feb-b0cd91b6a7cc

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = 0c9521da-2b10-45d0-aab6-4548ba625a8c
OK
Time taken: 2.808 seconds


In [18]:
%%bash
$HIVE_HOME/bin/hive -e "
CREATE TABLE default.amaht_geolocation (
truckid String,driverid String,event String,latitude String,longitude String,
city String,state String,velocity int,event_ind int,idling_ind int)
COMMENT 'geolocation'
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
LINES TERMINATED BY '\n'
STORED AS TEXTFILE
TBLPROPERTIES('skip.header.line.count'='1');
"

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = 1684157e-5175-491a-8577-c9320bf93503

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = dc24a10e-5e3d-451f-aa55-199e90e84616
OK
Time taken: 3.396 seconds


In [19]:
!$HIVE_HOME/bin/hive -e "show tables;"

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = 076f504d-ad93-4d32-9d5d-d5a920deb7ac

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = 7f487653-1791-4054-83cc-63faf23c18b9
OK
amaht_geolocation
amaht_trucks
Time taken: 1.572 seconds, Fetched: 2 row(s)


In [21]:
!$HIVE_HOME/bin/hive -e "DESCRIBE default.amaht_trucks;"

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = 4a9df482-0189-473c-b64c-a7d5677afd09

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = 848fb779-cffb-43f6-9ede-78b6985429c2
OK
driverid            	string              	                    
truckid             	string              	                    
model               	string              	                    
jun13_miles         	int                 	           

In [22]:
!$HIVE_HOME/bin/hive -e "DESCRIBE default.amaht_geolocation;"

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = 1b2b67f2-5cfd-4966-8fd4-0dba81ec5758

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = 81536293-9ebb-46b8-a35e-129870e02bbc
OK
truckid             	string              	                    
driverid            	string              	                    
event               	string              	                    
latitude            	string              	           

In [23]:
!$HADOOP_HOME/bin/hadoop fs -ls /user/hive/warehouse

Found 2 items
drwxr-xr-x   - root root       4096 2025-03-03 01:47 /user/hive/warehouse/amaht_geolocation
drwxr-xr-x   - root root       4096 2025-03-03 01:47 /user/hive/warehouse/amaht_trucks


In [24]:
!$HIVE_HOME/bin/hive -e "LOAD DATA LOCAL INPATH 'trucks_extended.csv' INTO TABLE default.amaht_trucks;"

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = 1d90b6d4-d569-4e37-baa6-30c6dbe3dd50

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = face4fbe-0a36-4cd3-beb4-d3859badd963
Loading data to table default.amaht_trucks
OK
Time taken: 4.289 seconds


In [26]:
!$HIVE_HOME/bin/hive -e "LOAD DATA LOCAL INPATH 'geolocation.csv' INTO TABLE default.amaht_geolocation;"

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = 55f4aef9-2735-49e4-9049-695681af5185

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = 518186d2-e40c-41eb-94a5-3f8bb5a7017e
Loading data to table default.amaht_geolocation
OK
Time taken: 2.955 seconds


In [27]:
!$HIVE_HOME/bin/hive -e "SELECT * FROM default.amaht_trucks LIMIT 1;"

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = 9fda9048-2312-4f18-bd0d-1cce749a832a

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = 9d1f189e-2e2a-4b53-aa96-f14c51e6a068
OK
A1	A1	Freightliner	9217	1914	8769	1892	14234	3008	11519	2262	8676	1596	10025	1878	12647	2331	10214	2054	10807	2134	11127	2191	9754	1967	12925	2578	15792	3313	9052	1878	11062	2150	13594	2824	11019	2324	9222	1902	8565	1745	10410	1943	13450	3047

In [28]:
!$HIVE_HOME/bin/hive -e "SELECT * FROM default.amaht_geolocation LIMIT 5;"

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = 6f8306c4-877a-40cd-a800-b815b7e3fc26

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = 1998bcdd-596d-408c-8996-6fc28dc9e3e6
OK
A54	A54	normal	38.440467	-122.714431	Santa Rosa	California	17	0	0
A20	A20	normal	36.977173	-121.899402	Aptos	California	27	0	0
A40	A40	overspeed	37.957702	-121.29078	Stockton	California	77	1	0
A31	A31	normal	39.409608	-123.355566	Willits	Cali

In [29]:
%%bash
$HIVE_HOME/bin/hive -e "
CREATE TABLE default.amaht_truck_mileage STORED AS ORC AS
SELECT truckid, driverid, rdate, miles, gas, miles / gas mpg
FROM default.amaht_trucks
LATERAL VIEW stack(54,
'jun13',jun13_miles,jun13_gas,'may13',may13_miles,may13_gas,
'apr13',apr13_miles,apr13_gas,'mar13',mar13_miles,mar13_gas,
'feb13',feb13_miles,feb13_gas,'jan13',jan13_miles,jan13_gas,
'dec12',dec12_miles,dec12_gas,'nov12',nov12_miles,nov12_gas,
'oct12',oct12_miles,oct12_gas,'sep12',sep12_miles,sep12_gas,
'aug12',aug12_miles,aug12_gas,'jul12',jul12_miles,jul12_gas,
'jun12',jun12_miles,jun12_gas,'may12',may12_miles,may12_gas,
'apr12',apr12_miles,apr12_gas,'mar12',mar12_miles,mar12_gas,
'feb12',feb12_miles,feb12_gas,'jan12',jan12_miles,jan12_gas,
'dec11',dec11_miles,dec11_gas,'nov11',nov11_miles,nov11_gas,
'oct11',oct11_miles,oct11_gas,'sep11',sep11_miles,sep11_gas,
'aug11',aug11_miles,aug11_gas,'jul11',jul11_miles,jul11_gas,
'jun11',jun11_miles,jun11_gas,'may11',may11_miles,may11_gas,
'apr11',apr11_miles,apr11_gas,'mar11',mar11_miles,mar11_gas,
'feb11',feb11_miles,feb11_gas,'jan11',jan11_miles,jan11_gas,
'dec10',dec10_miles,dec10_gas,'nov10',nov10_miles,nov10_gas,
'oct10',oct10_miles,oct10_gas,'sep10',sep10_miles,sep10_gas,
'aug10',aug10_miles,aug10_gas,'jul10',jul10_miles,jul10_gas,
'jun10',jun10_miles,jun10_gas,'may10',may10_miles,may10_gas,
'apr10',apr10_miles,apr10_gas,'mar10',mar10_miles,mar10_gas,
'feb10',feb10_miles,feb10_gas,'jan10',jan10_miles,jan10_gas,
'dec09',dec09_miles,dec09_gas,'nov09',nov09_miles,nov09_gas,
'oct09',oct09_miles,oct09_gas,'sep09',sep09_miles,sep09_gas,
'aug09',aug09_miles,aug09_gas,'jul09',jul09_miles,jul09_gas,
'jun09',jun09_miles,jun09_gas,'may09',may09_miles,may09_gas,
'apr09',apr09_miles,apr09_gas,'mar09',mar09_miles,mar09_gas,
'feb09',feb09_miles,feb09_gas,'jan09',jan09_miles,jan09_gas )
dummyalias AS rdate, miles, gas;
"

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = 7b4b1cc0-928f-4674-946a-be957678e66a

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = a02bd3dc-15b7-4c19-b97c-fb5f7ab2b8a5
Query ID = root_20250303015630_c434bc02-851b-4d42-8220-4a962397522c
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks is set to 0 since there's no reduce operator
Job running in-process (local Hadoop)
2025-03-03 01:56:35,793 Stage-1

In [30]:
!$HIVE_HOME/bin/hive -e "show tables;"

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = 56b2a109-533c-4aca-ae89-fb062c368c0d

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = f03aa3b2-c9d1-4034-9110-7f45d0cc6170
OK
amaht_geolocation
amaht_truck_mileage
amaht_trucks
Time taken: 1.593 seconds, Fetched: 3 row(s)


In [31]:
!$HIVE_HOME/bin/hive -e "DESCRIBE default.amaht_truck_mileage;"

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = 21f75a93-6a6c-412f-8ed0-3660c41f9c82

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = 2b2f40ef-5199-40d5-b310-b74e288f8eb4
OK
truckid             	string              	                    
driverid            	string              	                    
rdate               	string              	                    
miles               	int                 	           

In [32]:
!$HIVE_HOME/bin/hive -e "SET hive.cli.print.header=true;"
!$HIVE_HOME/bin/hive -e "SELECT * FROM default.amaht_truck_mileage LIMIT 10;"

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = a4d5f0b8-993a-4cf3-af42-1a900e483f00

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = 1fe8308f-f56c-4b5a-bd50-f9b44a3d1c08
SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.

In [33]:
%%bash
$HIVE_HOME/bin/hive -e "
CREATE TABLE default.amaht_avg_mileage
STORED AS ORC
AS
SELECT truckid, avg(mpg) avgmpg
FROM default.amaht_truck_mileage
GROUP BY truckid
DISTRIBUTE BY avgmpg
SORT BY avgmpg DESC;
"

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = 5fbb0df7-4cf4-4f8b-9853-afe0663d6b5e

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = 6cdc6d41-e771-42b3-bf20-26fe1c5d0c44
Query ID = root_20250303015925_841d1eac-78b1-4ec2-97a1-a7b76ca1574e
Total jobs = 2
Launching Job 1 out of 2
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  s

In [34]:
!$HIVE_HOME/bin/hive -e "SELECT * FROM default.amaht_avg_mileage LIMIT 10;"

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = bce22336-77db-469d-aa90-419c65a8dff2

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = eb5de300-7bf3-452a-8e69-17b50779bf34
OK
A18	6.295206724472384
A61	6.238571478738324
A79	6.230205355202399
A30	6.202007499297939
A60	5.96129692348809
A38	5.917100754501856
A13	5.8205454797114795
A40	5.8199316380329105
A2	5.795686564164179
A92	5.787692965941835
Time taken: 5.283 seco

In [35]:
%%bash
$HIVE_HOME/bin/hive -e "
CREATE EXTERNAL TABLE IF NOT EXISTS default.amaht_driver_mileage (
    truckid INT,
    driverid INT,
    miles INT,
    fuel FLOAT,
    mpg FLOAT
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
STORED AS TEXTFILE
LOCATION '/content/';"

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = 7dc4db08-1ae2-471c-b772-ad7dc8a3c4bd

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = 04a1d25c-8a11-48be-a2c7-f47e505f7cfb
OK
Time taken: 4.085 seconds


In [36]:
%%bash
$HIVE_HOME/bin/hive -e "
CREATE TABLE IF NOT EXISTS default.amaht_driver_mileage_agg
STORED AS ORC
AS
SELECT driverid, SUM(miles) AS total_miles
FROM default.amaht_driver_mileage
GROUP BY driverid
SORT BY total_miles DESC;
"

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = f3a16cb7-0831-43ee-8bea-c3dc2de63643

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = 8e0e4ec3-700d-4892-ad5a-301179892641
Query ID = root_20250303020211_1fea522f-bcde-4f8e-9f95-1017c8c92ba9
Total jobs = 2
Launching Job 1 out of 2
Number of reduce tasks not specified. Estimated from input data size: 16
In order to change the average load for a reducer (in bytes):
  

In [37]:
%%bash
$HIVE_HOME/bin/hive -e "
SELECT * FROM default.amaht_driver_mileage_agg LIMIT 10;"

1000000000	1000000000
100000000	100000000
10000000	10000000
16744206	2924588
1000000	1000000
6	300006
12313	134134
99999	134134
100000	110000
40960	81922


SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = 9cf2eade-a568-43aa-b150-46f21fa5497e

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = cb95b287-278d-42cf-8b86-87f4c0a25ad7
OK
Time taken: 5.121 seconds, Fetched: 10 row(s)


In [38]:
%%bash
$HIVE_HOME/bin/hive -e "
DESCRIBE FORMATTED default.amaht_driver_mileage_agg;"


# col_name            	data_type           	comment             
driverid            	int                 	                    
total_miles         	bigint              	                    
	 	 
# Detailed Table Information	 	 
Database:           	default             	 
OwnerType:          	USER                	 
Owner:              	root                	 
CreateTime:         	Mon Mar 03 02:03:46 UTC 2025	 
LastAccessTime:     	UNKNOWN             	 
Retention:          	0                   	 
Location:           	file:/user/hive/warehouse/amaht_driver_mileage_agg	 
Table Type:         	MANAGED_TABLE       	 
Table Parameters:	 	 
	COLUMN_STATS_ACCURATE	{\"BASIC_STATS\":\"true\"}
	bucketing_version   	2                   
	numFiles            	1                   
	numRows             	93                  
	rawDataSize         	672                 
	totalSize           	733                 
	transient_lastDdlTime	1740967427          
	 	 
# Storage Information	 	 
SerDe Library:     

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = a5d9b023-c02e-49f1-bbb1-c94b2ebe4082

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = 27e5c9d7-738f-40c1-96ae-65bb08a88fcf
OK
Time taken: 2.469 seconds, Fetched: 32 row(s)


In [39]:
%%bash
$HIVE_HOME/bin/hive -e "
SELECT driverid, total_miles
FROM default.amaht_driver_mileage_agg
ORDER BY total_miles DESC
LIMIT 10;
"

1000000000	1000000000
100000000	100000000
10000000	10000000
16744206	2924588
1000000	1000000
6	300006
99999	134134
12313	134134
100000	110000
40960	81922


SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = 6b22fca3-a3e9-44da-85f1-816893feb9c0

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = c3da197b-e4bd-4e7c-b0bc-47942226e82f
Query ID = root_20250303020813_c0eb475f-1ea3-4c31-9b58-3c57f749e684
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks determined at compile time: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reduce


###Hands-On Experiment 2-2:Data Warehousing with Hive
### 2. Enhanced Aggregation with Grouping Sets (0.5 points)
### Ex2-2
### 2.1 Dimensional Aggregation

In [40]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://downloads.apache.org/hadoop/common/hadoop-3.3.6/hadoop-3.3.6.tar.gz
!tar -xzf hadoop-3.3.6.tar.gz
!tar xf apache-hive-3.1.3-bin.tar
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["HADOOP_HOME"] = "/content/hadoop-3.3.6"
os.environ["HIVE_HOME"] = "/content/apache-hive-3.1.3-bin"

--2025-03-03 02:09:25--  https://downloads.apache.org/hadoop/common/hadoop-3.3.6/hadoop-3.3.6.tar.gz
Resolving downloads.apache.org (downloads.apache.org)... 135.181.214.104, 88.99.208.237, 2a01:4f9:3a:2c57::2, ...
Connecting to downloads.apache.org (downloads.apache.org)|135.181.214.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 730107476 (696M) [application/x-gzip]
Saving to: ‘hadoop-3.3.6.tar.gz.1’

hadoop-3.3.6.tar.gz 100%[===================>] 696.28M  6.36MB/s    in 21s     

2025-03-03 02:09:47 (32.5 MB/s) - ‘hadoop-3.3.6.tar.gz.1’ saved [730107476/730107476]



In [41]:
%%bash
$HIVE_HOME/bin/hive -e "
SELECT driverid, total_miles
FROM default.amaht_driver_mileage_agg
ORDER BY total_miles DESC
LIMIT 10;
"

1000000000	1000000000
100000000	100000000
10000000	10000000
16744206	2924588
1000000	1000000
6	300006
99999	134134
12313	134134
100000	110000
40960	81922


SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = 340351cc-9337-40cd-91da-d25600330aa8

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = b03b02cb-016a-4909-bd24-0a6138129ad1
Query ID = root_20250303021118_bfca8d82-91df-4894-9aeb-f3aa7db9d0cc
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks determined at compile time: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reduce

In [42]:
!$HIVE_HOME/bin/hive -e 'show databases;'

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = dcc700ad-ff0d-4aff-887b-c8ab6d66b1ea

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = 3ca5c9ef-bd24-41e4-be54-1b839302575e
OK
default
Time taken: 1.522 seconds, Fetched: 1 row(s)


In [43]:
!$HIVE_HOME/bin/hive -e 'use default;'

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = effb8183-f31f-4ab2-84df-cb466e887118

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = 13d3b7ed-ea1f-4e1b-ad1e-c06f4e5dc081
OK
Time taken: 1.548 seconds


In [44]:
!$HIVE_HOME/bin/hive -e 'show tables;'

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = 9e979ffe-cc7e-478c-9873-bfbc034b563d

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = 44bd5d1e-4f23-40f8-a7fb-984a400b14c6
OK
amaht_avg_mileage
amaht_driver_mileage
amaht_driver_mileage_agg
amaht_geolocation
amaht_truck_mileage
amaht_trucks
Time taken: 1.59 seconds, Fetched: 6 row(s)


In [45]:
!$HIVE_HOME/bin/hive -e "LOAD DATA LOCAL INPATH 'geolocation.csv' INTO TABLE default.amaht_geolocation;"

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = 6f8e0c71-0c9d-445e-8f7b-7b689c00a088

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = b194f7b3-bfeb-4b42-9cda-a7600417819e
Loading data to table default.amaht_geolocation
OK
Time taken: 2.878 seconds


In [46]:
## Calculate how many events occurred according to the event types, e.g., normal, overspeed,lane departure, etc., to each driver

%%bash
$HIVE_HOME/bin/hive -e "
SELECT driverid, event, count(*)
FROM default.amaht_geolocation
GROUP BY driverid, event;
"

A1	lane departure	4
A1	normal	154
A1	unsafe tail distance	2
A10	lane departure	2
A10	normal	148
A10	overspeed	2
A10	unsafe following distance	8
A100	lane departure	2
A100	normal	152
A100	unsafe following distance	4
A100	unsafe tail distance	2
A11	lane departure	2
A11	normal	150
A11	overspeed	2
A11	unsafe following distance	6
A12	lane departure	2
A12	normal	150
A12	overspeed	2
A12	unsafe following distance	4
A12	unsafe tail distance	2
A13	lane departure	2
A13	normal	152
A13	overspeed	4
A13	unsafe following distance	2
A14	lane departure	2
A14	normal	148
A14	overspeed	4
A14	unsafe following distance	4
A14	unsafe tail distance	2
A15	lane departure	4
A15	normal	148
A15	unsafe following distance	6
A15	unsafe tail distance	2
A16	lane departure	2
A16	normal	154
A16	overspeed	2
A16	unsafe following distance	2
A17	lane departure	2
A17	normal	154
A17	overspeed	4
A18	lane departure	8
A18	normal	148
A18	overspeed	2
A18	unsafe tail distance	2
A19	normal	160
A2	lane departure	2
A2	normal	158
A20	lane

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = c7df6a13-d136-443b-8404-61b7598bb8c8

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = 604641b7-529e-41b7-8d4a-15dff70c5e5b
Query ID = root_20250303021443_0bbf811b-689f-42e9-a35a-13afb4166b02
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  s

In [47]:
# Calculate total events that occurred per drivers
%%bash
$HIVE_HOME/bin/hive -e "
SELECT driverid, count(*)
FROM default.amaht_geolocation
GROUP BY driverid;
"

A1	160
A10	160
A100	160
A11	160
A12	160
A13	160
A14	160
A15	160
A16	160
A17	160
A18	160
A19	160
A2	160
A20	160
A21	160
A22	160
A23	160
A24	160
A25	160
A26	160
A27	160
A28	160
A29	160
A3	160
A30	160
A31	160
A32	160
A33	160
A34	160
A35	160
A36	160
A37	160
A38	160
A39	160
A4	160
A40	160
A41	160
A42	160
A43	160
A44	160
A45	160
A46	160
A47	160
A48	160
A49	160
A5	160
A50	160
A51	160
A52	160
A53	160
A54	160
A55	160
A56	160
A57	160
A58	160
A59	160
A6	160
A60	160
A61	160
A62	160
A63	160
A64	160
A65	160
A66	160
A67	160
A68	160
A69	160
A7	160
A70	160
A71	160
A72	160
A73	160
A74	160
A75	160
A76	160
A77	160
A78	160
A79	160
A8	160
A80	160
A81	160
A82	160
A83	160
A84	160
A85	160
A86	160
A87	160
A88	160
A89	160
A9	160
A90	160
A91	160
A92	160
A93	160
A94	160
A95	160
A96	160
A97	160
A98	160
A99	160


SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = b332eaed-86b1-4f32-a46c-286ccf08bb14

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = 93ca68d2-cb6a-4ff9-ac02-530f02f1edcc
Query ID = root_20250303021533_c7ce5eee-d25e-42d3-835d-60f3c15364f8
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  s


###However, you may experience errors if you wanted to do this operation in colab, due to the system limitations. Instead, we can create two temporary tables, store the query results in them, and then unionthem. This is a useful little hack that would help you, when you are dealing with even larger datasets


In [51]:
# Connecting both GROUP BY result sets with UNION ALL
%%bash
$HIVE_HOME/bin/hive -e "
CREATE TEMPORARY TABLE temp1 AS
SELECT driverid, event, count(*)
FROM default.amaht_geolocation
GROUP BY driverid, event;

CREATE TEMPORARY TABLE temp2 AS
SELECT driverid, CAST(null AS STRING) as event, count(*)
FROM default.amaht_geolocation
GROUP BY driverid;
SELECT * FROM temp1
UNION ALL
SELECT * FROM temp2;
"

A1	NULL	160
A10	NULL	160
A100	NULL	160
A11	NULL	160
A12	NULL	160
A13	NULL	160
A14	NULL	160
A15	NULL	160
A16	NULL	160
A17	NULL	160
A18	NULL	160
A19	NULL	160
A2	NULL	160
A20	NULL	160
A21	NULL	160
A22	NULL	160
A23	NULL	160
A24	NULL	160
A25	NULL	160
A26	NULL	160
A27	NULL	160
A28	NULL	160
A29	NULL	160
A3	NULL	160
A30	NULL	160
A31	NULL	160
A32	NULL	160
A33	NULL	160
A34	NULL	160
A35	NULL	160
A36	NULL	160
A37	NULL	160
A38	NULL	160
A39	NULL	160
A4	NULL	160
A40	NULL	160
A41	NULL	160
A42	NULL	160
A43	NULL	160
A44	NULL	160
A45	NULL	160
A46	NULL	160
A47	NULL	160
A48	NULL	160
A49	NULL	160
A5	NULL	160
A50	NULL	160
A51	NULL	160
A52	NULL	160
A53	NULL	160
A54	NULL	160
A55	NULL	160
A56	NULL	160
A57	NULL	160
A58	NULL	160
A59	NULL	160
A6	NULL	160
A60	NULL	160
A61	NULL	160
A62	NULL	160
A63	NULL	160
A64	NULL	160
A65	NULL	160
A66	NULL	160
A67	NULL	160
A68	NULL	160
A69	NULL	160
A7	NULL	160
A70	NULL	160
A71	NULL	160
A72	NULL	160
A73	NULL	160
A74	NULL	160
A75	NULL	160
A76	NULL	160
A77	NULL	160
A78	NULL	160
A79	N

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = a7e378c1-e840-4322-aa7a-7afaecd33a98

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = aa1d5e4f-f9fe-46b5-a872-4a79b88f970c
Query ID = root_20250303022423_d705f82c-fd34-4ea2-98d7-f8294f99abdc
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  s

### 2.2 GROUPING SETS

In [52]:
#Using GROUPING SETS
%%bash
$HIVE_HOME/bin/hive -e "
SELECT driverid, event, count(*) as occurance
FROM default.amaht_geolocation
GROUP BY driverid, event
GROUPING SETS ((driverid, event), driverid);
"

A1	NULL	160
A1	lane departure	4
A1	normal	154
A1	unsafe tail distance	2
A10	NULL	160
A10	lane departure	2
A10	normal	148
A10	overspeed	2
A10	unsafe following distance	8
A100	NULL	160
A100	lane departure	2
A100	normal	152
A100	unsafe following distance	4
A100	unsafe tail distance	2
A11	NULL	160
A11	lane departure	2
A11	normal	150
A11	overspeed	2
A11	unsafe following distance	6
A12	NULL	160
A12	lane departure	2
A12	normal	150
A12	overspeed	2
A12	unsafe following distance	4
A12	unsafe tail distance	2
A13	NULL	160
A13	lane departure	2
A13	normal	152
A13	overspeed	4
A13	unsafe following distance	2
A14	NULL	160
A14	lane departure	2
A14	normal	148
A14	overspeed	4
A14	unsafe following distance	4
A14	unsafe tail distance	2
A15	NULL	160
A15	lane departure	4
A15	normal	148
A15	unsafe following distance	6
A15	unsafe tail distance	2
A16	NULL	160
A16	lane departure	2
A16	normal	154
A16	overspeed	2
A16	unsafe following distance	2
A17	NULL	160
A17	lane departure	2
A17	normal	154
A17	overspeed	4
A18	NU

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = 86e05b12-a8a6-4884-8265-e54873807380

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = 1e97e388-8534-4f06-9e85-c3acea2c5641
Query ID = root_20250303022549_9fbe2703-da8b-4e8d-b172-d8be13ccb980
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  s

### 3 Analyzing Driver Risk factor (0.5 points)
### 3.1 Non-normal events

In [53]:
%%bash
$HIVE_HOME/bin/hive -e "
CREATE TABLE default.amaht_unusual_events
STORED AS ORC
AS
SELECT driverid, count(*) as occurence
FROM default.amaht_geolocation
WHERE event != 'normal'
GROUP BY driverid;
"

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = ff395c7f-f54e-4adb-bba4-bf5d6b1d5a04

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = 21816c85-7e89-4f6a-bb32-fa7581686f14
Query ID = root_20250303022910_72d3af1a-200d-4479-b20b-c168fe274e49
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  s

### The resulting table will have a count of total unusual or non-normal events associated with each driver.

In [54]:
%%bash
$HIVE_HOME/bin/hive -e "
SELECT *
FROM default.amaht_unusual_events
LIMIT 5;
"

A1	6
A10	12
A100	8
A11	10
A12	10


SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = a05344f3-c5e5-411f-8358-d33245a0c260

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = 7a272cda-69e1-46a1-ab7f-1449590ec829
OK
Time taken: 4.603 seconds, Fetched: 5 row(s)


In [55]:
%%bash
$HIVE_HOME/bin/hive -e "SHOW TABLES;"

amaht_avg_mileage
amaht_driver_mileage
amaht_driver_mileage_agg
amaht_geolocation
amaht_truck_mileage
amaht_trucks
amaht_unusual_events


SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = ed8a43b0-5703-47fe-9a50-3311a9e14196

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = 17d55c6d-391b-4f61-92d0-1feedc8c637b
OK
Time taken: 2.659 seconds, Fetched: 7 row(s)


### 3.2 Perform JOIN operation
Let’s create the driver_total_mileage table. The driver_total_mileage table has the total miles
traveled by each driver:

In [56]:
# Creating driver_total_mileage table
%%bash
$HIVE_HOME/bin/hive -e "
CREATE TABLE default.amaht_driver_total_mileage
STORED AS ORC
AS
SELECT driverid, sum(miles) totalmiles
FROM default.amaht_truck_mileage
GROUP BY driverid
DISTRIBUTE BY totalmiles
SORT BY totalmiles DESC;
"

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = 2966812c-7145-4997-987a-f54d91cc4920

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = 2d81aad4-241c-4b42-a13a-83a13b8fb47d
Query ID = root_20250303023152_5f6319d5-07ec-47d7-9be0-65b19e56229c
Total jobs = 2
Launching Job 1 out of 2
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  s

In [57]:
%%bash
$HIVE_HOME/bin/hive -e "SHOW TABLES;"


amaht_avg_mileage
amaht_driver_mileage
amaht_driver_mileage_agg
amaht_driver_total_mileage
amaht_geolocation
amaht_truck_mileage
amaht_trucks
amaht_unusual_events


SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = b72682e4-497d-42c1-8570-9fa1de93f63f

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = 97c65cb6-cd93-4314-b27c-4bb36873e5b2
OK
Time taken: 1.556 seconds, Fetched: 8 row(s)


### NOTE: As you remember from the previous hands-on exercise, we used a trick to reduce the load on our colab instance, by using DISTRIBUTE BY ... and SORT BY .... In a normal setting, we only needed to use ORDER BY ...Next, we will perform a JOIN operation. The unusual_events table has the driver’s ID and count of their respective non-normal events

In [58]:
# Creating joined table
%%bash
$HIVE_HOME/bin/hive -e "
CREATE TABLE default.amaht_joined
STORED AS ORC
AS
SELECT a.driverid, a.occurence, b.totalmiles
FROM amaht_unusual_events a
JOIN amaht_driver_total_mileage b
ON (a.driverid = b.driverid);
"

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = de0374d4-9054-42c0-81d1-7b9c193ee847

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = e286d6dd-bf23-4e4c-8c83-de9ca107cdda
Query ID = root_20250303023413_9c042ec0-aea4-413a-9da4-62f0a422d408
Total jobs = 1
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See htt

The resulting data set will give us a driver’s total miles and the number of non-normal events.

In [59]:
#Display results
%%bash
$HIVE_HOME/bin/hive -e "
SELECT *
FROM default.amaht_joined
LIMIT 5;
"

A52	4	684131
A80	10	681115
A94	12	676750
A5	18	676574
A10	12	675377


SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = efffbb19-5a99-4dde-9470-d268e9d0be7a

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = 005d7a78-56d2-4f49-9036-958174a3365b
OK
Time taken: 5.274 seconds, Fetched: 5 row(s)


In [ ]:
## 4 Rollup and CUBE (3 points)

In [60]:
%%bash
$HIVE_HOME/bin/hive -e "
CREATE TABLE IF NOT EXISTS default.amaht_unusual_events
STORED AS ORC
AS
SELECT driverid, count(*) as occurence
FROM default.amaht_geolocation
WHERE event != 'normal'
GROUP BY driverid;
"

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = 7079415c-52e7-462c-b6ab-82995196596d

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = 54a0265f-11fc-4eeb-ad8f-eb2428066584
OK
Time taken: 2.538 seconds


In [61]:
%%bash
$HIVE_HOME/bin/hive -e "
SELECT *
FROM default.amaht_unusual_events
LIMIT 5;
"

A1	6
A10	12
A100	8
A11	10
A12	10


SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = f5e2e082-fc7e-4286-8474-6f39916cac9a

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = 351cfd7c-838d-436c-aec1-a3b1d0b2da8c
OK
Time taken: 4.082 seconds, Fetched: 5 row(s)


In [62]:
%%bash
$HIVE_HOME/bin/hive -e "SELECT * from default.amaht_trucks limit 5;"

A1	A1	Freightliner	9217	1914	8769	1892	14234	3008	11519	2262	8676	1596	10025	1878	12647	2331	10214	2054	10807	2134	11127	2191	9754	1967	12925	2578	15792	3313	9052	1878	11062	2150	13594	2824	11019	2324	9222	1902	8565	1745	10410	1943	13450	3047	10446	2112	10378	2201	9014	1598	9466	2642	14175	2642	11573	2642	13055	2642	14614	2642	14099	2642	15174	2642	10635	2642	12105	2642	10130	2642	11344	2642	10622	2642	10747	2642	12976	2642	13221	2642	10435	2642	12839	2642	8969	2642	8795	2642	13668	2642	12414	2642	13300	2642	11698	2642	14207	2642	14062	2642	12073	2642	15276	2642	9880	2642	13861	2642	11176	2642
A2	A2	Ford	12058	2335	14314	2648	11050	2323	14114	3157	13583	2346	15362	3353	13608	2607	11236	2597	11380	1939	12934	2324	11848	2301	13206	3114	15316	3344	13838	2718	12024	2800	12693	2329	14392	2929	11134	2290	13911	2953	11542	2650	11296	2135	10224	2027	14595	3013	13769	2980	10377	1827	12786	1827	14364	1827	8978	1827	9259	1827	9545	1827	13100	1827	8529	1827	9667	1827	13264	1827	12070	1827	10367	18

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = 05ac36e8-24cd-4ce3-9eb1-77093b4d3938

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = 7cfa4a4e-724c-4fd1-b0f2-4e28c6e16cdf
OK
Time taken: 4.022 seconds, Fetched: 5 row(s)


In [63]:
!$HIVE_HOME/bin/hive -e "LOAD DATA LOCAL INPATH 'trucks_extended.csv' INTO TABLE default.default.amaht_trucks;"

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = e89ed931-689d-499a-ae37-6fbd9ea65ae1

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = aa865485-a05a-4967-80c8-8e5ae466f2ad
FAILED: ParseException line 1:71 missing EOF at '.' near 'default'


In [64]:
%%bash
$HIVE_HOME/bin/hive -e "
CREATE TABLE IF NOT EXISTS default.amaht_truck_mileage STORED AS ORC AS
SELECT truckid, driverid, rdate, miles, gas, miles / gas mpg
FROM default.amaht_trucks
LATERAL VIEW stack(54,
'jun13',jun13_miles,jun13_gas,'may13',may13_miles,may13_gas,
'apr13',apr13_miles,apr13_gas,'mar13',mar13_miles,mar13_gas,
'feb13',feb13_miles,feb13_gas,'jan13',jan13_miles,jan13_gas,
'dec12',dec12_miles,dec12_gas,'nov12',nov12_miles,nov12_gas,
'oct12',oct12_miles,oct12_gas,'sep12',sep12_miles,sep12_gas,
'aug12',aug12_miles,aug12_gas,'jul12',jul12_miles,jul12_gas,
'jun12',jun12_miles,jun12_gas,'may12',may12_miles,may12_gas,
'apr12',apr12_miles,apr12_gas,'mar12',mar12_miles,mar12_gas,
'feb12',feb12_miles,feb12_gas,'jan12',jan12_miles,jan12_gas,
'dec11',dec11_miles,dec11_gas,'nov11',nov11_miles,nov11_gas,
'oct11',oct11_miles,oct11_gas,'sep11',sep11_miles,sep11_gas,
'aug11',aug11_miles,aug11_gas,'jul11',jul11_miles,jul11_gas,
'jun11',jun11_miles,jun11_gas,'may11',may11_miles,may11_gas,
'apr11',apr11_miles,apr11_gas,'mar11',mar11_miles,mar11_gas,
'feb11',feb11_miles,feb11_gas,'jan11',jan11_miles,jan11_gas,
'dec10',dec10_miles,dec10_gas,'nov10',nov10_miles,nov10_gas,
'oct10',oct10_miles,oct10_gas,'sep10',sep10_miles,sep10_gas,
'aug10',aug10_miles,aug10_gas,'jul10',jul10_miles,jul10_gas,
'jun10',jun10_miles,jun10_gas,'may10',may10_miles,may10_gas,
'apr10',apr10_miles,apr10_gas,'mar10',mar10_miles,mar10_gas,
'feb10',feb10_miles,feb10_gas,'jan10',jan10_miles,jan10_gas,
'dec09',dec09_miles,dec09_gas,'nov09',nov09_miles,nov09_gas,
'oct09',oct09_miles,oct09_gas,'sep09',sep09_miles,sep09_gas,
'aug09',aug09_miles,aug09_gas,'jul09',jul09_miles,jul09_gas,
'jun09',jun09_miles,jun09_gas,'may09',may09_miles,may09_gas,
'apr09',apr09_miles,apr09_gas,'mar09',mar09_miles,mar09_gas,
'feb09',feb09_miles,feb09_gas,'jan09',jan09_miles,jan09_gas )
dummyalias AS rdate, miles, gas;
"

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = 0f11b3bd-a8b7-4a05-ab08-0788e449acb4

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = 05b4f15e-220e-4eb7-aaf2-492d8ec1bae0
OK
Time taken: 1.861 seconds


In [65]:
!$HIVE_HOME/bin/hive -e "show tables;"

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = 6b4e8c03-d653-46b1-ae87-e997167e945e

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = 6a826c09-41bf-4a3a-a833-06d0fd6949ec
OK
amaht_avg_mileage
amaht_driver_mileage
amaht_driver_mileage_agg
amaht_driver_total_mileage
amaht_geolocation
amaht_joined
amaht_truck_mileage
amaht_trucks
amaht_unusual_events
Time taken: 1.718 seconds, Fetched: 9 row(s)


In [66]:
%%bash
$HIVE_HOME/bin/hive -e "
CREATE TABLE IF NOT EXISTS default.amaht_driver_total_mileage
STORED AS ORC
AS
SELECT driverid, sum(miles) totalmiles
FROM default.amaht_truck_mileage
GROUP BY driverid
DISTRIBUTE BY totalmiles
SORT BY totalmiles DESC;
"

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = ee0de5c4-871e-489b-8ab7-d0bc56cd189f

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = 9c664a1a-7f56-4c64-a831-1013cad203f2
OK
Time taken: 1.861 seconds


In [67]:
%%bash
$HIVE_HOME/bin/hive -e "
CREATE TABLE IF NOT EXISTS default.amaht_joined
STORED AS ORC
AS
SELECT a.driverid, a.occurence, b.totalmiles
FROM amaht_unusual_events a
JOIN amaht_driver_total_mileage b
ON (a.driverid = b.driverid);
"

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = 1bb8360f-130b-4c1b-ac44-48217e9ce356

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = 323b0752-70a9-4077-9c37-1611871fa89c
OK
Time taken: 2.869 seconds


In [68]:
%%bash
$HIVE_HOME/bin/hive -e "
SELECT *
FROM default.amaht_joined
LIMIT 5;
"

A52	4	684131
A80	10	681115
A94	12	676750
A5	18	676574
A10	12	675377


SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = fcffa430-77e9-4b30-976f-a309153544ee

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = f7d8be27-2cd8-41e7-8688-45cabe5ad9d5
OK
Time taken: 4.057 seconds, Fetched: 5 row(s)


###3.3 Compute driver risk factor
We will associate a driver risk factor with every driver. The risk factor for each driver is the number of
abnormal occurrences over the total number of miles per driver. Simply put, a high number of abnormal
occurrences over a short number of miles driven is an indicator of high risk.
Let’s translate this intuition into an HiveQL query. NOTE: I believe by now, you should have a
good handle on this, but please remember that you need to add the necessary bash and HIVE_HOME
directives before the following commands, just like previous prompts!

In [69]:
%%bash
$HIVE_HOME/bin/hive -e  "SELECT driverid, occurence, totalmiles, totalmiles/occurence AS riskfactor
FROM default.amaht_joined
ORDER BY riskfactor ASC;"

A97	40	631199	15779.975
A73	20	642222	32111.1
A35	18	637069	35392.72222222222
A50	18	640069	35559.38888888889
A5	18	676574	37587.444444444445
A95	16	642920	40182.5
A71	16	672026	42001.625
A92	14	618049	44146.357142857145
A86	14	629558	44968.42857142857
A44	14	652599	46614.21428571428
A77	14	663403	47385.92857142857
A49	12	627335	52277.916666666664
A98	12	634665	52888.75
A69	12	637123	53093.583333333336
A3	12	639584	53298.666666666664
A43	12	640233	53352.75
A81	12	645273	53772.75
A93	12	645930	53827.5
A14	12	649645	54137.083333333336
A38	12	650497	54208.083333333336
A15	12	651036	54253.0
A84	12	651620	54301.666666666664
A75	12	652379	54364.916666666664
A60	12	653030	54419.166666666664
A34	12	655656	54638.0
A18	12	656474	54706.166666666664
A66	12	660413	55034.416666666664
A4	12	663289	55274.083333333336
A87	12	665023	55418.583333333336
A9	12	665456	55454.666666666664
A10	12	675377	56281.416666666664
A94	12	676750	56395.833333333336
A51	10	622764	62276.4
A48	10	631133	63113.3
A20	10	63149

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = 183c6839-176a-47a5-8682-b00943a32d6b

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = 1d33ef5c-5f16-467d-bcf4-19989caabd73
Query ID = root_20250303024622_17b7aace-c90b-4c5d-981a-c4424b7e4e04
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks determined at compile time: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reduce

In [70]:
%%bash
$HIVE_HOME/bin/hive -e  "SELECT driverid, occurence, totalmiles, totalmiles/occurence AS riskfactor
FROM default.amaht_joined
ORDER BY riskfactor ASC LIMIT 11;"

A97	40	631199	15779.975
A73	20	642222	32111.1
A35	18	637069	35392.72222222222
A50	18	640069	35559.38888888889
A5	18	676574	37587.444444444445
A95	16	642920	40182.5
A71	16	672026	42001.625
A92	14	618049	44146.357142857145
A86	14	629558	44968.42857142857
A44	14	652599	46614.21428571428
A77	14	663403	47385.92857142857


SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = d5c12808-e0a5-4889-9627-a13d4d88c192

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = 198708bf-d44e-42d2-bbc3-cf71c2ea444f
Query ID = root_20250303024752_a57a4690-f5a0-4348-80bd-102dcbd1637c
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks determined at compile time: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reduce

In [72]:
# HiveQL query, Let us call it grouping-set-query. Remember that you need to add the necessary bash and HIVE_HOME directives before the following commands,
%%bash
$HIVE_HOME/bin/hive -e "SELECT driverid, event, city, count(*) as occurance
FROM default.amaht_geolocation
GROUP BY driverid, event, city
GROUPING SETS ((driverid, event, city), (driverid, event), driverid);"

A1	NULL	NULL	160
A1	lane departure	NULL	4
A1	lane departure	Aptos	2
A1	lane departure	Santa Rosa	2
A1	normal	NULL	154
A1	normal	Antelope	4
A1	normal	Aptos	8
A1	normal	Arbuckle	10
A1	normal	Gilroy	14
A1	normal	Klamath	4
A1	normal	Lodi	14
A1	normal	Markleeville	12
A1	normal	Modesto	14
A1	normal	Napa	4
A1	normal	Oceano	14
A1	normal	Palmdale	20
A1	normal	San Diego	14
A1	normal	San Dimas	12
A1	normal	Santa Rosa	10
A1	unsafe tail distance	NULL	2
A1	unsafe tail distance	Markleeville	2
A10	NULL	NULL	160
A10	lane departure	NULL	2
A10	lane departure	Stockton	2
A10	normal	NULL	148
A10	normal	Apple Valley	6
A10	normal	Goleta	8
A10	normal	Isleton	4
A10	normal	Kneeland	14
A10	normal	La Puente	14
A10	normal	Palo Alto	28
A10	normal	San Dimas	12
A10	normal	San Quentin	22
A10	normal	Santa Paula	12
A10	normal	Stockton	12
A10	normal	Willits	16
A10	overspeed	NULL	2
A10	overspeed	Apple Valley	2
A10	unsafe following distance	NULL	8
A10	unsafe following distance	Goleta	2
A10	unsafe following distance	Palo Alt

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = 37ed33d1-bd02-4bd3-9f9e-189072b73a43

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = f890834e-3801-4e7f-8aff-a2b7f4289975
Query ID = root_20250303025019_2d873f35-9eda-4228-951e-a4b99ed3ca56
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  s

 Q1. (1pt) Modify/rewrite the grouping-set-query in the example with ROLLUP (Let’s call it
rollup-query). Run it, check the results, and explain the differences.

– Replace the GROUPING SETS part with ROLLUP:

∗ Delete GROUPING SETS line

∗ Add ‘WITH ROLLUP’ at the end of the GROUP BY line.
For syntax, see this Apache Hive cwiki link

– Show query results. The first line of the result should be like below

∗ NULL NULL NULL 8000

– Explain the differences in the results between the grouping-set-query and the roll-up-query
with your query results.

Q2. (1pt) Modify/rewrite the rollup-query (or grouping-set-query) with GROUPING SETS
(Let’s call it rollup-like-query) to produce identical results of rollup-query

– The query result should be the same.

– Show query results

• Q3. (1pt) Modify/rewrite the grouping-set-query with CUBE. Run it, check the results, and
explain the differences and similarities.

– Explain the differences and similarities of the query results among GROUPING SETS,
ROLLUP, and CUBE

– Show query results
7


###Q1. (1pt) Modify/rewrite the grouping-set-query in the example with ROLLUP (Let’s call it rollup-query). Run it, check the results, and explain the differences.

– Replace the GROUPING SETS part with ROLLUP:

∗ Delete GROUPING SETS line

∗ Add ‘WITH ROLLUP’ at the end of the GROUP BY line. For syntax, see this Apache Hive cwiki link

– Show query results. The first line of the result should be like below

∗ NULL NULL NULL 8000

– Explain the differences in the results between the grouping-set-query and the roll-up-query with your query results.

Solutions:

### Explanation of Differences:

> Aggregation Style:

>GROUPING SETS: Allows custom combinations of groupings. You specify exactly which combinations of columns you want aggregated.

>ROLLUP: Creates a hierarchical summary, starting from the most detailed grouping and rolling up to more general groupings automatically.

>GROUPING SETS gives us custom combinations of aggregations — like exactly the sets

>ROLLUP automatically creates all higher-level summaries in a hierarchical order.

>In the ROLLUP result, the first row (NULL, NULL, NULL) represents the grand total, showing the overall count across all groups.

In [73]:
%%bash
$HIVE_HOME/bin/hive -e " SELECT driverid, event, city, count(*) as occurance
FROM default.amaht_geolocation
GROUP BY driverid, event, city WITH ROLLUP;
"

NULL	NULL	NULL	16000
A1	NULL	NULL	160
A1	lane departure	NULL	4
A1	lane departure	Aptos	2
A1	lane departure	Santa Rosa	2
A1	normal	NULL	154
A1	normal	Antelope	4
A1	normal	Aptos	8
A1	normal	Arbuckle	10
A1	normal	Gilroy	14
A1	normal	Klamath	4
A1	normal	Lodi	14
A1	normal	Markleeville	12
A1	normal	Modesto	14
A1	normal	Napa	4
A1	normal	Oceano	14
A1	normal	Palmdale	20
A1	normal	San Diego	14
A1	normal	San Dimas	12
A1	normal	Santa Rosa	10
A1	unsafe tail distance	NULL	2
A1	unsafe tail distance	Markleeville	2
A10	NULL	NULL	160
A10	lane departure	NULL	2
A10	lane departure	Stockton	2
A10	normal	NULL	148
A10	normal	Apple Valley	6
A10	normal	Goleta	8
A10	normal	Isleton	4
A10	normal	Kneeland	14
A10	normal	La Puente	14
A10	normal	Palo Alto	28
A10	normal	San Dimas	12
A10	normal	San Quentin	22
A10	normal	Santa Paula	12
A10	normal	Stockton	12
A10	normal	Willits	16
A10	overspeed	NULL	2
A10	overspeed	Apple Valley	2
A10	unsafe following distance	NULL	8
A10	unsafe following distance	Goleta	2
A10	unsafe follow

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = b07bc3dc-9994-4220-9c4d-424871279301

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = ad4fa7ff-b9d3-40c6-9c16-1205b9d733e3
Query ID = root_20250303025826_583711fd-3621-49b1-9a70-6e5fda07f3ea
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  s

### Q2. (1pt) Modify/rewrite the rollup-query (or grouping-set-query) with GROUPING SETS (Let’s call it rollup-like-query) to produce identical results of rollup-query

– The query result should be the same.

– Show query results

Solutions:

The results of the GROUPING SETS query are identical to the ROLLUP query. Both show:

>Detailed breakdown by driver, event, and city

>Higher-level roll-ups, removing city and event step by step

>A grand total row aggregating all records

In [74]:
%%bash
$HIVE_HOME/bin/hive -e " SELECT driverid, event, city, count(*) as occurance
FROM default.amaht_geolocation
GROUP BY driverid, event, city
GROUPING SETS (
  (driverid, event, city),
  (driverid, event),
  driverid,
  ()
);"

NULL	NULL	NULL	16000
A1	NULL	NULL	160
A1	lane departure	NULL	4
A1	lane departure	Aptos	2
A1	lane departure	Santa Rosa	2
A1	normal	NULL	154
A1	normal	Antelope	4
A1	normal	Aptos	8
A1	normal	Arbuckle	10
A1	normal	Gilroy	14
A1	normal	Klamath	4
A1	normal	Lodi	14
A1	normal	Markleeville	12
A1	normal	Modesto	14
A1	normal	Napa	4
A1	normal	Oceano	14
A1	normal	Palmdale	20
A1	normal	San Diego	14
A1	normal	San Dimas	12
A1	normal	Santa Rosa	10
A1	unsafe tail distance	NULL	2
A1	unsafe tail distance	Markleeville	2
A10	NULL	NULL	160
A10	lane departure	NULL	2
A10	lane departure	Stockton	2
A10	normal	NULL	148
A10	normal	Apple Valley	6
A10	normal	Goleta	8
A10	normal	Isleton	4
A10	normal	Kneeland	14
A10	normal	La Puente	14
A10	normal	Palo Alto	28
A10	normal	San Dimas	12
A10	normal	San Quentin	22
A10	normal	Santa Paula	12
A10	normal	Stockton	12
A10	normal	Willits	16
A10	overspeed	NULL	2
A10	overspeed	Apple Valley	2
A10	unsafe following distance	NULL	8
A10	unsafe following distance	Goleta	2
A10	unsafe follow

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = 9845af18-951a-48a8-8bd0-bb405649cb05

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = b0d3bed3-05dd-40e7-ba45-76ba2c278d18
Query ID = root_20250303030723_1ca93f7f-c607-42dd-93c9-088d25cc5aff
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  s

### Q3. (1pt) Modify/rewrite the grouping-set-query with CUBE. Run it, check the results, and explain the differences and similarities.

– Explain the differences and similarities of the query results among GROUPING SETS, ROLLUP, and CUBE

– Show query results 7

Solutions:

### Here is a step-by-step breakdown comparing GROUPING SETS, ROLLUP, and CUBE in HiveQL:

1. Aggregation Style

>GROUPING SETS: Here we can use our  own column set to group by, allowing us to control how we roll up data.

>ROLLUP: It provides a hierarchical aggregation of the data starting at the most detailed level and rolling up progressively to more aggregate levels.

>CUBE: It generates all possible combinations of the specified columns, creating a complete aggregation of the data.

2. Output Levels

>GROUPING SETS: we specify particular sets of groupings. The result will only have the given combinations.

>ROLLUP: The result contains the whole hierarchy from the lowest level (detailed group) to the highest level (aggregate), with a last row for the total sum.

>CUBE: The outcome includes all column combinations for aggregation, together with a sum row total.

3. Grand Total Row

>GROUPING SETS: Only a grand total row is shown if we explicitly include it in your query.

>ROLLUP: A grand total row is always included, at the highest level of aggregation.

>CUBE: A grand total row is always added to the result.

4. Flexibility

>GROUPING SETS: Offers very high flexibility since we can define our own column combinations to group by.

>ROLLUP: Offers medium flexibility since the hierarchy is fixed, and the output depends on a sequential roll-up.

>CUBE: Offers the highest flexibility since it computes all possible group combinations of the columns you specify.

5. Use Case

>GROUPING SETS: Suitable for ad-hoc summaries where we need particular, custom group combinations.

>ROLLUP: Ideal to create a step-by-step roll-up or a hierarchy of sums, e.g., a sales sum by region and further by country, city, etc.

>CUBE: Best suited to an exhaustive dimension breakdown where we need to compare all combinations of attributes (e.g., all product-region combination of sales).




In [75]:
%%bash
$HIVE_HOME/bin/hive -e "SELECT driverid, event, city, count(*) as occurance
FROM default.amaht_geolocation
GROUP BY driverid, event, city with CUBE;"

NULL	NULL	NULL	16000
NULL	NULL	Antelope	630
NULL	NULL	Apple Valley	630
NULL	NULL	Aptos	472
NULL	NULL	Arbuckle	472
NULL	NULL	Bakersfield	316
NULL	NULL	Cloverdale	156
NULL	NULL	Fresno	158
NULL	NULL	Gilroy	158
NULL	NULL	Glen Ellen	158
NULL	NULL	Glendora	158
NULL	NULL	Goleta	158
NULL	NULL	Hollister	158
NULL	NULL	Holtville	158
NULL	NULL	Homeland	158
NULL	NULL	Irvine	316
NULL	NULL	Isleton	316
NULL	NULL	Jacumba	316
NULL	NULL	Klamath	158
NULL	NULL	Kneeland	158
NULL	NULL	Knights Landing	158
NULL	NULL	Knightsen	158
NULL	NULL	La Palma	316
NULL	NULL	La Puente	158
NULL	NULL	La Quinta	158
NULL	NULL	Lodi	312
NULL	NULL	Mariposa	156
NULL	NULL	Markleeville	156
NULL	NULL	Martinez	156
NULL	NULL	Marysville	156
NULL	NULL	Modesto	316
NULL	NULL	Napa	158
NULL	NULL	Occidental	156
NULL	NULL	Oceano	156
NULL	NULL	Oceanside	156
NULL	NULL	Ojai	156
NULL	NULL	Palmdale	334
NULL	NULL	Palo Alto	334
NULL	NULL	Palo Cedro	334
NULL	NULL	Redding	158
NULL	NULL	Roseville	158
NULL	NULL	Rough and Ready	158
NULL	NULL	San Diego	314

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-3.3.6/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = 74b92298-51a6-4f9f-870b-b462a69db106

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = 34a24c0c-3b03-4612-be41-3ff845dc3093
Query ID = root_20250303031859_c4702675-c7ee-485c-b724-5bedc394745e
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  s